In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"

In [4]:
!pip install transformers
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece

     |████████████████████████████████| 2.3MB 8.1MB/s 
     |████████████████████████████████| 3.3MB 58.0MB/s 
     |████████████████████████████████| 901kB 63.7MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32913 sha256=e06f8fa9be7d9e3e892f69b3c2312ac881552b416b2dbca633fbe004f605528c
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 2.6MB 8.0MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-cp37-none-any.whl size=7054 sha256=e3fdd406fd5323c200463d110c34d7a2c34f52961238cfec416b1be0b470331c
  Stored in directory: /root/.cache/pip/wheels/7d/37/fa/2098a4aa6c0d94d6ddff0d3a79669e12bc4f7baca8a760b3db
Successfully built language-detector
     |████████████████████████████████| 1.2MB 7.0MB/s 


In [7]:
import sys
sys.path.append(root_path)

In [8]:
import tensorflow as tf

from src.encoders.context_encoder_bert import ContextEncoder

from tensorflow.python import keras
import toml
import json
import pandas as pd

from src.dataset.distilbert import DistilBERTDataset

## Experiment

In [9]:
sys.path.append(root_path)
from utils.experiments import get_experiments, save_results

In [10]:
# Read local `config.toml` file.
config = get_experiments('DISTILBERT_FINETUNE_TEST')
config_df = pd.DataFrame.from_dict(config)
config_df

,bert_type,dataset_type,finetune_bert,pct_data,augment_pct,epochs
0,distilbert,clinical,True,1,0.1,100
1,distilbert,clinical,True,1,0.5,100
2,distilbert,clinical,True,1,1.0,100
3,distilbert,fiction,True,1,0.1,100
4,distilbert,fiction,True,1,0.5,100
5,distilbert,fiction,True,1,1.0,100
6,distilbert,wiki,True,1,0.1,100
7,distilbert,wiki,True,1,0.5,100
8,distilbert,wiki,True,1,1.0,100


In [11]:
sys.path.append(root_path)
config_df.to_csv(f'{root_path}/models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [14]:
for experiment in config:
    bert_type = experiment['bert_type']
    dataset_type = experiment['dataset_type']
    finetune_bert = experiment['finetune_bert']
    pct_data = experiment['pct_data']
    augment_pct = experiment['augment_pct']
    epochs = experiment['epochs']
    BATCH_SIZE = 32
    print("params:", bert_type, dataset_type, finetune_bert, pct_data, augment_pct, epochs)
    
    # init model
    print("initializing model...")
    model = ContextEncoder(final_dropout=0.5,
                           dense_neurons=64,
                           bert_trainable=finetune_bert,
                           bert_type="distilbert-base-cased")
    
    # init dataset
    print("initializing dataset...")
    dataset = DistilBERTDataset(dataset_type=dataset_type,
                           pct_data=pct_data,
                            max_seq_length=128,
                           max_segment_length=5,
                           augment_pct=augment_pct)
    
    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()
    
    # create checkpoint path
    checkpoint_filepath = '{}/models/DistilBERT/finetune/simple/{}-{}-{}-pct-{}-aug/checkpoint'.format(
                            root_path,
                            dataset_type,                    
                            len(sentences), 
                            pct_data,
                            augment_pct)
    print(checkpoint_filepath)
    
    # get callbacks ready.
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        save_best_only=False,
        mode="auto",
        save_freq="epoch")

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy', 
        verbose=1,
        patience=10,
        mode='max',
        restore_best_weights=True)

    callbacks = [
    #     early_stopping,
        model_checkpoint_callback
    ]
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[
                      keras.metrics.BinaryAccuracy(name='accuracy')
                  ])
    
    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")
    
    # 
    print("starting the training process...")
    history = model.fit(dataset.format_sentences_tri_input(tokenized_sentences), 
                        tf.convert_to_tensor(labels), 
                        epochs=epochs,
                        validation_split=0.1,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        # class_weight=class_weight,
                        callbacks=callbacks)
    
    # assigning history to experiment object for saving.
    experiment["history"] = history.history
    
    print("saving results...")
    save_results(experiment)

params: distilbert clinical True 1 0.1 100
initializing model...
initializing dataset...
processing dataset...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/models/DistilBERT/finetune/simple/clinical-5484-1-pct-0.1-aug/checkpoint
compiling the model...
No checkpoint available.
starting the training process...
Epoch 1/100
155/155 [==============================] - 127s 706ms/step - loss: 0.6733 - accuracy: 0.7364 - val_loss: 0.5428 - val_accuracy: 0.7814
Epoch 2/100
155/155 [==============================] - 106s 681ms/step - loss: 0.5865 - accuracy: 0.7550 - val_loss: 0.5305 - val_accuracy: 0.7814
Epoch 3/100
155/155 [==============================] - 106s 681ms/step - loss: 0.5722 - accuracy: 0.7660 - val_loss: 0.5371 - val_accuracy: 0.7814
Epoch 4/100
155/155 [==============================] - 106s 681ms/step - loss: 0.5622 - accuracy: 0.7751 - val_loss: 0.5251 - val_accuracy: 0.7814
Epoch 5/100
155/155 [==============================] - 106s 681ms/step - loss: 0.5465 - accuracy: 0.7805 - val_loss: 0.5295 - val_accuracy: 0.7814
Epoch 6/100
155/155 [==============================] 

TypeError: ignored